# 🧩 `parse_markdown_examples_to_json(text: str)`

Парсит текст с примерами, содержащими блоки `**Input:**` и `**Output:**`, и возвращает их в виде словаря. Этот формат используется у LeetCode.

---

## 📥 Аргументы

| Параметр | Тип   | Описание                             |
|----------|-------|--------------------------------------|
| `text`   | `str` | Текст, содержащий примеры в формате: |
| -        |       | `**Example N:**`                     |
| -        |       | `**Input:** <входные данные>`        |
| -        |       | `**Output:** <выходные данные>`      |

---

## 📤 Возвращает

Тип: `dict`

Структура возвращаемого словаря:

```json
{
  "examples": [
    {
      "input": {...},     // Входные данные (dict)
      "output": ...       // Выходные данные (любой тип)
    }
  ]
}
```

---


# 🔻 Вставьте пример для конвертирования в JSON

In [1]:
example_block = """
**Example 1:**

![](https://assets.leetcode.com/uploads/2021/07/23/lc-midlist1.jpg)

**Input:** head = [1,2,3,4,5]
**Output:** [3,4,5]
**Explanation:** The middle node of the list is node 3.

**Example 2:**

![](https://assets.leetcode.com/uploads/2021/07/23/lc-midlist2.jpg)

**Input:** head = [1,2,3,4,5,6]
**Output:** [4,5,6]
**Explanation:** Since the list has two middle nodes with values 3 and 4, we return the second one.
"""

## 1. Функция-обработчик

In [2]:
import re
import ast
import json

def parse_markdown_examples_to_json(text: str):
    examples = []
    # Разделяем текст на блоки по шаблону **Example N:**
    raw_blocks = re.split(r"\*\*Example \d+:\*\*", text)
    for block in raw_blocks:
        # Ищем **Input:** ... **Output:** в каждом блоке
        input_match = re.search(r"\*\*Input:\*\*\s*(.*?)\s*(?:\n|$)", block)
        output_match = re.search(r"\*\*Output:\*\*\s*(.*?)\s*(?:\n|$)", block)

        if not input_match or not output_match:
            continue

        input_text = input_match.group(1).strip()
        output_text = output_match.group(1).strip().rstrip(",")

        # Парсинг входных данных (input)
        try:
            input_expr = "{" + re.sub(r"(\w+)\s*=", r'"\1":', input_text) + "}"
            input_dict = ast.literal_eval(input_expr)
        except Exception as e:
            print(f"⚠ Ошибка в input '{input_text}': {e}")
            continue

        # Подготовка output-текста
        output_text = output_text.replace("true", "True").replace("false", "False")

        # Парсинг выходных данных (output)
        try:
            output = ast.literal_eval(output_text)
        except Exception as e:
            print(f"⚠ Ошибка в output '{output_text}': {e}")
            continue

        # Добавляем пример в список
        examples.append({
            "input": input_dict,
            "output": output
        })

    return {"examples": examples}



## 2. Парсим примеры из текста

In [3]:
parsed = parse_markdown_examples_to_json(example_block)
print(parsed)

{'examples': [{'input': {'head': [1, 2, 3, 4, 5]}, 'output': [3, 4, 5]}, {'input': {'head': [1, 2, 3, 4, 5, 6]}, 'output': [4, 5, 6]}]}


## 3. Преобразуем результат в JSON-строку с красивым форматированием

In [4]:
pretty = json.dumps(parsed, indent=2, ensure_ascii=False)
print(pretty)

{
  "examples": [
    {
      "input": {
        "head": [
          1,
          2,
          3,
          4,
          5
        ]
      },
      "output": [
        3,
        4,
        5
      ]
    },
    {
      "input": {
        "head": [
          1,
          2,
          3,
          4,
          5,
          6
        ]
      },
      "output": [
        4,
        5,
        6
      ]
    }
  ]
}


## 4. Убираем лишние пробелы и переносы строк внутри массивов

In [5]:
pretty = re.sub(r'\[\s+([^\[\]]+?)\s+\]', lambda m: '[' + m.group(1).replace('\n', '').replace(' ', '') + ']', pretty)
print(pretty)

{
  "examples": [
    {
      "input": {
        "head": [1,2,3,4,5]
      },
      "output": [3,4,5]
    },
    {
      "input": {
        "head": [1,2,3,4,5,6]
      },
      "output": [4,5,6]
    }
  ]
}


# 🔺 Заберите результат в буфер обмена